### Euraxess scraping

In [49]:
# Import libraries
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [50]:
# Change the search_wrds by the domain yoiu want
search_wrds = ['bioinformatics','computational biology']

In [51]:
# number of pages to retrieve for each search word
pages = range(20)
domain = 'phd'

In [52]:
def extract_ids(search_1,page):
    # Set the URL you want to webscrape from
    if page ==0:
        url = 'https://euraxess.ec.europa.eu/site/search/type/offer_posting?keywords='+search_1+'%20'+domain+'&sort=changed&order=desc'
    else:   
        url = 'https://euraxess.ec.europa.eu/site/search/type/offer_posting?keywords='+search_1+'%20'+domain+'&sort=changed&order=desc&page='+str(page)
    # Connect to the URL
    response = requests.get(url)
    # Parse HTML and save to BeautifulSoup object¶
    soup = BeautifulSoup(response.text, "html.parser")
    ids = soup.findAll('h2')[3:]
    ids = [int(str(id_).split('/jobs/')[1].split('"')[0]) for id_ in ids]
    return ids

In [53]:
def extract_job_metadata(url_):
    response_ = requests.get(url_)
    soup_ = BeautifulSoup(response_.text, "html.parser")
    x = ' '.join(soup_.html.findAll(text=True,recursive=True)) 
    #Position Name
    pos_name = x.split('Jobs & Funding  ›')[1].split('\n \n \n')[0][1:-1]
    # Deadline
    deadline = str(soup_.findAll('h4')[1]).split('Deadline: ')[1].split(' -')[0]
    day = deadline.split(' ')[0]
    time = deadline.split(' ')[1]
    # Institute
    y = x.split('ORGANISATION/COMPANY')
    institute = y[1].split(' \n ')[2][5:-6]
    # loaction
    y = x.split('LOCATION')
    location = y[1].split(' \n ')[2][5:-6]
    country = location.split(' › ')[0]
    if location.count('›') ==1:
        city = location.split(' › ')[1]
    else:
        city = ' '
    # determine job describtion
    describtion = x.split('\n \n \n \n \n \n \n \n')[19]#.split('EURAXESS offer ID')[0]#.split('\n \n \n \n \n \n \n \n')[1]
    #starting date
    if x.count('OFFER STARTING DATE') >0:
        y = x.split('OFFER STARTING DATE')
        offering_date = y[1].split('\n')[2]
    else:
        offering_date = 'UNDEFINED'
    #full text
    html = str(x)
    return pos_name,day, time, institute, country,city,offering_date,describtion,html

In [55]:
def build_job_datafarme(search_wrds):
    columns = ['Euraxess_Link','Name','Deadline_day','Deadline_time','institute','country','city','offering_date',
               'describtion','html','search']
    final_df = pd.DataFrame(columns= columns)
    for search_term in search_wrds:
        search_1 = search_term.replace(' ', '%20') 
        i = 0        
        df = pd.DataFrame(columns= columns,index=range(10*len(pages)))
        df['search'] = search_term
        for page in pages:
            ids = extract_ids(search_1,page)
            
            for id_ in ids:
                url_ = 'https://euraxess.ec.europa.eu/jobs/'+str(id_)
                df.loc[i,'Euraxess_Link'] = url_
                df.loc[i,['Name','Deadline_day','Deadline_time','institute','country','city','offering_date'
                          ,'describtion','html']] = extract_job_metadata(url_)
                i +=1
        final_df = pd.concat([final_df,df])
    return final_df

In [ ]:
final_df = build_job_datafarme(search_wrds)

In [ ]:
final_df = final_df.dropna()

In [ ]:
final_df_ = final_df.drop_duplicates(subset=['Euraxess_Link','Name','Deadline_day','Deadline_time',
                                             'institute','country','city','describtion','html'])
from datetime import datetime
#final_df_['day'] = pd.to_datetime(final_df_['day'],format='%d/%m/%y')
final_df_.day = final_df_.day.apply(lambda x: datetime.strptime(x, "%d/%m/%Y"))
final_df_= final_df_.sort_index(by=['Deadline_day'])
final_df_.to_csv('PhD_Scraping.csv')